In [1]:
import requests
from bs4 import BeautifulSoup
import csv

def fetch_page_content(url):
    """Verilen URL'den sayfa içeriğini getir."""
    response = requests.get(url)
    return BeautifulSoup(response.content, 'html.parser')

def extract_headlines(soup):
    """Gelen BeautifulSoup nesnesinden haber başlıklarını çıkar."""
    return soup.find_all('div', class_='govde')

def extract_news_content(news_url):
    """Haber URL'sinden içeriği çıkar."""
    news_soup = fetch_page_content(news_url)
    # 'bottom' sınıfına sahip div'leri hariç tut
    bottom_div = news_soup.find('div', class_='bottom')
    if bottom_div:
        bottom_div.extract()  # 'bottom' div'ini çıkar

    paragraphs = news_soup.find_all('p')
    # Sadece haberle alakalı içerikleri filtrele
    news_content = ' '.join(
        p.text.strip() for p in paragraphs if p.text.strip() and 
        p.parent.name != 'div' and 
        'feed-comment' not in p.parent.get('class', [])
    )
    return news_content

def save_to_csv(filename, headlines):
    """Haber başlıkları ve içeriklerini verilen dosyaya kaydet."""
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Haber Başlığı', 'Haber İçeriği'])  # Başlık satırı ekle
        writer.writerows(headlines)

def main():
    """Ana işlem akışı."""
    page_number = 1
    max_page_number = 10
    total_headlines = []

    while page_number <= max_page_number:
        url = f'https://www.donanimhaber.com/kripto-para?sayfa={page_number}'
        soup = fetch_page_content(url)
        headlines = extract_headlines(soup)
        
        if not headlines:  # Eğer başlık yoksa döngüyü sonlandır
            break

        for headline in headlines:
            title_tag = headline.find('a', class_='baslik')  # 'baslik' sınıfına sahip başlık
            if title_tag:
                news_title = title_tag.text.strip()  # Başlık metni
                news_url = 'https://www.donanimhaber.com' + title_tag['href']  # Tam bağlantıyı oluştur
                news_content = extract_news_content(news_url)  # İçeriği çıkar

                if news_content:  # Eğer içerik boşsa kaydetme
                    total_headlines.append([news_title, news_content])  # Başlık ve içeriği listeye ekle

        page_number += 1  # Sonraki sayfaya geç

    # Sonuçları CSV'ye kaydet
    save_to_csv('donanim_haber.csv', total_headlines)
    print(f"{len(total_headlines)} haber başlığı ve içeriği 'donanim_haber.csv' dosyasına kaydedildi.")

if __name__ == "__main__":
    main()


200 haber başlığı ve içeriği 'donanim_haber.csv' dosyasına kaydedildi.
